## Circle, triangle and rectangle
- Hard mode

In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf
tf.autograph.set_verbosity(0)
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)

seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)
import warnings
warnings.filterwarnings(action = 'ignore')

### Creating dataset

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1 / 255,
                                    rotation_range = 10,
                                    width_shift_range = 0.2,
                                    height_shift_range = 0.2,
                                    shear_range = 0.7,
                                    zoom_range = [0.9, 2.2],
                                    horizontal_flip = True,
                                    vertical_flip = True,
                                    fill_mode = 'nearest')

In [12]:
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
image = load_img('handwriting/hard_handwriting_shape/train/triangle/triangle001.png')
x = img_to_array(image)
x = x.reshape((1, ) + x.shape)

In [13]:
import os
if not os.path.exists('preview') :
    os.mkdir('preview')

i = 0
for batch in train_datagen.flow(x, batch_size = 1, save_to_dir = 'preview', save_prefix = 'tri', 
                                save_format = 'png') :
    i += 1
    if i > 20 :
        break

In [14]:
train_generator = train_datagen.flow_from_directory(
    'handwriting/hard_handwriting_shape/train',
    target_size = (24, 24),
    batch_size = 3,
    class_mode = 'categorical'
)

Found 45 images belonging to 3 classes.


In [15]:
test_datagen = ImageDataGenerator(rescale = 1 / 255.)
test_generator = test_datagen.flow_from_directory(
    'handwriting/hard_handwriting_shape/test',
    target_size = (24, 24),
    batch_size = 3,
    class_mode = 'categorical'
)

Found 15 images belonging to 3 classes.


In [16]:
train_generator.labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2], dtype=int32)

In [17]:
train_generator.filenames[0]

'circle/circle001.png'

### Processing (model defining / setting / learning)

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten

In [19]:
model = Sequential([
     Conv2D(32, (3, 3), input_shape = (24, 24, 3), activation = 'relu'),
     Conv2D(64, (3, 3), activation = 'relu'),
     MaxPooling2D(),
     Flatten(),
     Dense(128, activation = 'relu'),
     Dense(3, activation = 'softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 22, 22, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 10, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6400)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               819328    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 387       
Total params: 839,107
Trainable params: 839,107
Non-trainable params: 0
__________________________________________________

In [20]:
model.compile(
    optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy']
)

In [21]:
model.fit_generator(
    train_generator, steps_per_epoch = 15, epochs = 200,
    validation_data = test_generator, validation_steps = 5, verbose = 2)

00
15/15 - 0s - loss: 1.1022 - accuracy: 0.3111 - val_loss: 1.0791 - val_accuracy: 0.4667
Epoch 3/200
15/15 - 0s - loss: 1.0280 - accuracy: 0.4889 - val_loss: 1.1301 - val_accuracy: 0.4667
Epoch 4/200
15/15 - 0s - loss: 0.8228 - accuracy: 0.6000 - val_loss: 1.1417 - val_accuracy: 0.6000
Epoch 5/200
15/15 - 0s - loss: 0.6843 - accuracy: 0.7333 - val_loss: 1.4015 - val_accuracy: 0.6667
Epoch 6/200
15/15 - 0s - loss: 0.9131 - accuracy: 0.5778 - val_loss: 1.0586 - val_accuracy: 0.6667
Epoch 7/200
15/15 - 0s - loss: 0.7208 - accuracy: 0.7111 - val_loss: 1.4816 - val_accuracy: 0.4000
Epoch 8/200
15/15 - 0s - loss: 0.7521 - accuracy: 0.6222 - val_loss: 1.0515 - val_accuracy: 0.7333
Epoch 9/200
15/15 - 0s - loss: 0.6714 - accuracy: 0.7111 - val_loss: 1.0068 - val_accuracy: 0.7333
Epoch 10/200
15/15 - 0s - loss: 0.4836 - accuracy: 0.8667 - val_loss: 1.0679 - val_accuracy: 0.8000
Epoch 11/200
15/15 - 0s - loss: 0.4439 - accuracy: 0.8222 - val_loss: 1.0945 - val_accuracy: 0.7333
Epoch 12/200
15/1

In [22]:
model.evaluate(test_generator, steps = 5)

5/5 [==============================] - 0s 1ms/step - loss: 1.8443 - accuracy: 0.7333


[1.8442530632019043, 0.7333333492279053]